In [ ]:
from PIL import Image
import tempfile
import cv2
import numpy as np
import imutils

def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

def cont_canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    edged = cv2.adaptiveThreshold(edged,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    (cnts, contours, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:5]
    screenCnt = contours[0]
    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            screenCnt = approx
            break
    cv2.drawContours(image,[screenCnt], -1, (0, 255, 0), 2)
    return screenCnt
    

def order_points(pts):
    s = pts.sum(axis=1)
    rect=np.zeros((4,2),dtype='float32')
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([[0, 0],[maxWidth - 1, 0],[maxWidth - 1, maxHeight - 1],[0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def morph_bin(warped):
    #Sharpening of warped image
    _threshold=2
    blurred = cv2.GaussianBlur(warped, (5,5), 1.0)
    sharpened = 2.25 * warped - (1.35)* blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if _threshold > 0:
        low_contrast_mask = np.absolute(warped - blurred) < _threshold
        np.copyto(sharpened, warped, where=low_contrast_mask)
    warped=sharpened
    warped = warped-(np.full_like(warped, 10))
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5,5), np.uint8) 
    img_erosion = cv2.erode(warped, kernel, iterations=1)
    (thresh, warped) = cv2.threshold(warped, 95, 255, cv2.THRESH_BINARY)
    warped = cv2.GaussianBlur(warped,(5,5),0)
    warped = cv2.adaptiveThreshold(warped,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,1)
    denoised=cv2.fastNlMeansDenoising(warped, 3, 7, 21)
    cv2.imshow('Final',imutils.resize(denoised,height=500,width=500))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

ratio1=min(float(1024/Image.open("document_basic.png").size[0]),1)
ratio2=min(float(1024/Image.open("document_adv.png").size[0]),1)

img1_file_name=set_image_dpi("document_basic.png")
img2_file_name=set_image_dpi("document_adv.png")

img1=cv2.imread(img1_file_name)
screenCnt1=cont_canny(img1)
img2=cv2.imread(img2_file_name)
screenCnt2=cont_canny(img2)

pts1 = np.array(screenCnt1.reshape(4, 2) * ratio1)
pts2 = np.array(screenCnt2.reshape(4, 2) * ratio2)
warped1 = four_point_transform(np.asarray(Image.open("document_basic.png")), pts1)
warped2 = four_point_transform(np.asarray(Image.open("document_adv.png")), pts2)

morph_bin(warped1)
morph_bin(warped2)